# Generate IDS for rule checking

## Prerequisites

In [ ]:
%pip install ifcopenshell ifctester pystache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00


## Create IDS file using the python tools from ifcopenshell

In [6]:
import ifcopenshell
from ifctester import ids, reporter

#--------------------------------------------------
# create an IDS (0.9.7) or (1.0?)
#--------------------------------------------------
my_ids = ids.Ids(title="Accessibility Data verification",
                 purpose="Checking the Flemish accessibility regulation")

# ----------------------------------------
# # Add a specification to the stairs
my_spec = ids.Specification(
    name="Stairs may not be too steep",
    identifier="VTV Art.20 §2+§3",
    ifcVersion="IFC2X3")
my_spec.applicability.append(ids.Entity(name="IFCSTAIR"))

restriction = ids.Restriction(
    options={"maxInclusive": 17},
    base="integer")
property = ids.Property(
    propertySet="Pset_StairCommon",
    baseName="NumberOfRiser",
    cardinality="required",
    dataType="IfcCountMeasure",
    instructions="Art.20§2 The amount of risers can be at most 17",
    value=restriction
)
my_spec.requirements.append(property)

restriction = ids.Restriction(
    options={"maxInclusive": 0.18},
    base="length")
property = ids.Property(
    propertySet="Pset_StairCommon",
    baseName="RiserHeight",
    cardinality="required",
    dataType="IfcPositiveLengthMeasure",
    instructions="Art.20§3 Riser Height should be at most 18cm",
    value=restriction
)
my_spec.requirements.append(property)

restriction = ids.Restriction(
    options={"minInclusive": 0.23},
    base="length")
property = ids.Property(
    propertySet="Pset_StairCommon",
    baseName="TreadLength",
    cardinality="required",
    dataType="IfcPositiveLengthMeasure",
    instructions="Art.20§3 Tread Length should be at least 23cm",
    value=restriction
)
my_spec.requirements.append(property)

my_ids.specifications.append(my_spec)

# ----------------------------------------
# DOORS
my_spec = ids.Specification(
    name="Each door should have a height of at least 2.09m.",
    identifier="VTV Art.22 §1",
    ifcVersion="IFC2X3")
my_spec.applicability.append(ids.Entity(name="IFCDOOR"))
restriction = ids.Restriction(
    options={"minInclusive": 2090}, # mm
    base="length")
attribute = ids.Attribute(
    name="OverallHeight",
    cardinality="required",
    instructions="Art.22§1 Doors should be at least 2.09m high.",
    value=restriction
)
my_spec.requirements.append(attribute)
my_ids.specifications.append(my_spec)


my_spec = ids.Specification(
    name="Each door should have a width of at least 1.05m.",
    identifier="VTV Art.22 §2",
    ifcVersion="IFC2X3")
my_spec.applicability.append(ids.Entity(name="IFCDOOR"))
restriction = ids.Restriction(
    options={"minInclusive": 1050}, # 1050 mm
    base="length")
attribute = ids.Attribute(
    name="OverallWidth",
    cardinality="required",
    instructions="Art.22§2 Doors should be at least 1.05m wide.",
    value=restriction
)
my_spec.requirements.append(attribute)
my_ids.specifications.append(my_spec)

# ----------------------------------------
# save to file
result = my_ids.to_xml("VTV_accessibility.ids")

## Test the IFC file

In [7]:
# load IDS
# my_ids = ids.open("VTV_Accessibility_usbim.ids")

# open  IFC file
my_ifc = ifcopenshell.open("../datasets/heartbreak_hotel.ifc")

# validate IFC model against IDS requirements:
my_ids.validate(my_ifc)

# show results:
reporter.Console(my_ids).report()

Accessibility Data verification
[FAIL] (2/3) Stairs may not be too steep
    Applies to:
        All IFCSTAIR data
    Requirements:
        NumberOfRiser data shall be {'maxInclusive': 17} and in the dataset Pset_StairCommon
            The property value 18.0 does not match the requirements - #8870=IfcStair('1qnuHei2H9IuA8Q46m1Dkm',#18,'Precast Stair:Stair:383689',$,'Precast Stair:Precast Stair',#8869,$,'383689',.HALF_TURN_STAIR.)
        RiserHeight data shall be {'maxInclusive': 0.18} and in the dataset Pset_StairCommon
        TreadLength data shall be {'minInclusive': 0.23} and in the dataset Pset_StairCommon
[FAIL] (11/18) Each door should have a height of at least 2.09m.
    Applies to:
        All IFCDOOR data
    Requirements:
        The OverallHeight shall be {'minInclusive': 2090}
            The attribute value 2031.9999999999993 does not match the requirement - #13313=IfcDoor('2wcvH1AOD8fgqG2PBvpM7U',#18,'M_Single-Flush:0915 x 2032mm:398822',$,'M_Single-Flush:0915 x 2032

In [8]:
# show results in HTML
reporter_obj = reporter.Html(my_ids)
reporter_obj.report()
reporter_obj.to_file("report.html")

23718